# Model

In [ ]:
import torch
import torch.nn as nn

In [ ]:
k = torch.randn(2,3,4)
print(k.shape)

In [ ]:
k_class = nn.Linear(4, 2)
k_class.weight.data = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]], dtype=torch.float32)
k_class.bias.data = torch.tensor([1, 2], dtype=torch.float32)

opt = k_class(k[:, 1:,:])
print(opt.shape)

In [ ]:
from backbone import *

model = BERT(embed_dim = 768,
                 num_heads = 12,
                 expansion_factor = 4,
                 num_layers = 12,
                 max_seq_len = 256,
                 vocab_size = 12000)

model

# Data

### DownLoad

In [ ]:
# DownLoad
from modules import *

bookscorpus_download()

### Preprocess

In [ ]:
from modules import *

corpus_pth = './data/books_test'

txt_clean(corpus_pth)

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../tokenization')
sys.path.append('../tokenization/tools/')

from tokenization.tokenizer import *
from tokenization.tools import *

tokenizer = BPETokenizer()
tokenizer.load('../transformers/iwslt2013_tokenizer.pkl')

In [ ]:
def Produce_data_pair_corpus(corpus_path, tokenizer, max_seq_len=256):
    file_names = os.listdir(corpus_path)
    data_pair_corpus = []
    for file_name in file_names:
        print('Processing file: ', file_name)
        with open(os.path.join(corpus_path, file_name), 'r') as f:
            lines = f.readlines()
        for line in lines:
            tokens = tokenizer.tokenize(line)[0]
            num_token = len(tokens)
            num_pair = num_token // max_seq_len + 1
            for i in range(num_pair):
                if i == num_pair - 1:
                    percent = random.uniform(0.4, 0.6)
                    index = int(percent * (num_token - i * max_seq_len))
                    data_pair_corpus.append([tokens[i*max_seq_len:i*max_seq_len+index], tokens[i*max_seq_len+index:]])
                else:
                    percent = random.uniform(0.4, 0.6)
                    index = int(percent * max_seq_len)
                    data_pair_corpus.append((tokens[i*max_seq_len:i*max_seq_len+index], tokens[i*max_seq_len+index:(i+1)*max_seq_len]))

    return data_pair_corpus

In [ ]:
data_pair = produce_data_pair_corpus(
    corpus_path = './data/books_test/',
    tokenizer = tokenizer,
    max_seq_len = 256
)

In [ ]:
for pair in data_pair:
    print(len(pair[0]), len(pair[1]))
    print(pair)

In [ ]:
import pickle

file_pth = './data/books_test/data_pair_corpus.pkl'

with open(file_pth, 'wb') as f:
    pickle.dump(data_pair, f)

In [ ]:
from torch.utils.data import DataLoader
from modules import *

import sys
sys.path.append('../')
sys.path.append('../tokenization')
sys.path.append('../tokenization/tools/')

from tokenization.tokenizer import *
from tokenization.tools import *

tokenizer = BPETokenizer()
tokenizer.load('../transformers/iwslt2013_tokenizer.pkl')

import pickle

file_pth = './data/books_test/data_pair_corpus.pkl'

with open(file_pth, 'rb') as f:
    data_pair_corpus = pickle.load(f)


train_dataset = BERTDataset(
    data_pair_corpus = data_pair_corpus,
    tokenizer = tokenizer,
    mask_token = '[MASK]',
    cls_token = '[CLS]',
    pad_token = '[PAD]',
    seq_token = '[SEP]',
    max_seq_len = 256
)


dataloader = DataLoader(
    train_dataset,
    batch_size = 8,
    shuffle = True,
    num_workers = 4
)

In [ ]:
i = 1
for item in dataloader:
    i -= 1
    if i == 0:
        break